In [1]:
%load_ext dotenv
%dotenv

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import os

subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace = os.getenv("WORKSPACE")

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [3]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:input-data-automl:1")

In [ ]:
from azure.ai.ml import automl

classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

In [6]:

from azure.ai.ml.automl import ClassificationPrimaryMetrics

list(ClassificationPrimaryMetrics)

[<ClassificationPrimaryMetrics.AUC_WEIGHTED: 'AUCWeighted'>,
 <ClassificationPrimaryMetrics.ACCURACY: 'Accuracy'>,
 <ClassificationPrimaryMetrics.NORM_MACRO_RECALL: 'NormMacroRecall'>,
 <ClassificationPrimaryMetrics.AVERAGE_PRECISION_SCORE_WEIGHTED: 'AveragePrecisionScoreWeighted'>,
 <ClassificationPrimaryMetrics.PRECISION_SCORE_WEIGHTED: 'PrecisionScoreWeighted'>]

In [ ]:
classification_job.set_limits(
    timeout_minutes=60,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True,
)

In [ ]:
returned_job = ml_client.jobs.create_or_update(
    classification_job
)
aml_url = returned_job.studio_url